# CA3 - Data Driven Model
- Student Name: **Vinicius Moura Barros**
- Student Number: **T00244396**
## **Allocation of marks**

Stages 1,2,3   Total 30%

Ideally your dataset will not need a lot of work. Marks going for the choosing,  training, evaluation and refining of model primarily


---



Stage 4,5       Total 60%

Choose your model

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html


Train and evaluate your model. Refine the model or choose another estimator or approach?

Do not overwrite cells and add commentry between iterations.



---




Stage 6         Total 10%

# **Stage 1**
***Describe Data***

A brief description of the data, where you sourced it, headers of the data. Summary statistics, charts etc...

If you using data from different sources how you might link them.

```
pip install scikit-learn

```

 ### Importing the libraries

In [1]:
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

### Loading Dataset

In [2]:
DATASET = 'data/overwatch_match_map_stats.csv'
df = pd.read_csv(DATASET)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13901 entries, 0 to 13900
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   round_start_time              13901 non-null  object 
 1   round_end_time                13901 non-null  object 
 2   stage                         13901 non-null  object 
 3   match_id                      13901 non-null  int64  
 4   game_number                   13901 non-null  int64  
 5   match_winner                  13901 non-null  object 
 6   map_winner                    13901 non-null  object 
 7   map_loser                     13901 non-null  object 
 8   map_name                      13901 non-null  object 
 9   map_round                     13901 non-null  int64  
 10  winning_team_final_map_score  13901 non-null  int64  
 11  losing_team_final_map_score   13901 non-null  int64  
 12  control_round_name            4270 non-null   object 
 13  A

,round_start_time,round_end_time,stage,match_id,game_number,match_winner,map_winner,map_loser,map_name,map_round,...,team_one_name,team_two_name,attacker_payload_distance,defender_payload_distance,attacker_time_banked,defender_time_banked,attacker_control_perecent,defender_control_perecent,attacker_round_end_score,defender_round_end_score
0,01/11/18 00:12,01/11/18 00:20,2018: Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,1,...,Los Angeles Valiant,San Francisco Shock,75.61505,0.00000,0.000000,240.000,NaN,NaN,2,0
1,01/11/18 00:22,01/11/18 00:27,2018: Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,2,...,Los Angeles Valiant,San Francisco Shock,75.64960,75.61505,125.750570,0.000,NaN,NaN,3,2
2,01/11/18 00:34,01/11/18 00:38,2018: Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,1,...,Los Angeles Valiant,San Francisco Shock,0.00000,0.00000,250.492000,240.000,NaN,NaN,2,0
3,01/11/18 00:40,01/11/18 00:44,2018: Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,2,...,Los Angeles Valiant,San Francisco Shock,0.00000,0.00000,225.789030,250.492,NaN,NaN,2,2
4,01/11/18 00:46,01/11/18 00:49,2018: Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,3,...,Los Angeles Valiant,San Francisco Shock,0.00000,0.00000,36.396057,250.492,NaN,NaN,4,2


### Filtering the data
We don't need all the data, so we will filter it to only the data necessary to train the model


In [3]:
df = df[['team_one_name', 'team_two_name', 'map_name', 'map_winner', 'map_loser', 'winning_team_final_map_score',
         'losing_team_final_map_score']]

In [4]:
df.head()

,team_one_name,team_two_name,map_name,map_winner,map_loser,winning_team_final_map_score,losing_team_final_map_score
0,Los Angeles Valiant,San Francisco Shock,Dorado,Los Angeles Valiant,San Francisco Shock,3,3
1,Los Angeles Valiant,San Francisco Shock,Dorado,Los Angeles Valiant,San Francisco Shock,3,3
2,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,Los Angeles Valiant,San Francisco Shock,4,4
3,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,Los Angeles Valiant,San Francisco Shock,4,4
4,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,Los Angeles Valiant,San Francisco Shock,4,4


In [5]:
default_dict = defaultdict(LabelEncoder)

# we need to convert the categorical data into numerical data so the model can understand it
def categorize_df(df: pd.DataFrame):
    return df.apply(lambda x: default_dict[x.name].fit_transform(x))

def prettify_df(categorized_df: pd.DataFrame):
    return categorized_df.apply(lambda x: default_dict[x.name].inverse_transform(x))

def get_map_winner_name(map_winner: int):
    res = default_dict['map_winner'].inverse_transform([map_winner])
    return res.item(0)



In [6]:

# select the features and target variable
# features = ['team_one_name', 'team_two_name','map_name', 'map_winner', 'map_loser', 'winning_team_final_map_score', 'losing_team_final_map_score']
features = ['team_one_name', 'team_two_name', 'map_name']
target = 'map_winner'

categorized_df = categorize_df(df)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(categorized_df[features], categorized_df[target], test_size=0.5, random_state=42)

print(len(X_train), len(X_test), len(y_train), len(y_test))

# create a decision tree classifier
estimator = DecisionTreeClassifier()

# fit the model on the training data
estimator.fit(X_train, y_train)

# make predictions on the testing data
y_pred = estimator.predict(X_test)

# evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

6950 6951 6950 6951
Accuracy: 0.7221982448568551


In [7]:
to_test = categorized_df.loc[[3, 6, 9, 200, 450, 780, 1600]]

print(to_test[['team_one_name', 'team_two_name', 'map_winner']])
# test_df = pd.DataFrame(df[features])
# test_df = categorize_df(test_df)

prediction = estimator.predict(to_test[features])
print(prediction)
# for map_winner_id in prediction:
#     print(map_winner_id)
#     print(f"Winner is {get_map_winner_name(map_winner_id)}")


      team_one_name  team_two_name  map_winner
3                10             14          10
6                10             14          10
9                10             14          10
200               8              3           3
450               9              1           1
780              17              9           9
1600             14             13          14
[10 10 10  3  1  9 13]


# **Stage 2**

Justification for the property that you would like the model to predict.

# **Stage 3**

Cleaning the data, linking datasets etc.. 



# **Stage 4**
Choose and train Estimator.

# **Stage 5**
Evaluate your model and revisit 4...repeat..

# **Stage 6**
***Present results and Conclusions***

